In [ ]:
print("TAKEN FROM GEORG'S REPO, AND LUKES!")
import sys
sys.path.append("/workspaces/LMSimRecon/PETSIRD/python")
import array_api_compat.numpy as xp
import parallelproj
import matplotlib.pyplot as plt
from array_api_compat import to_device
from generate_fake_list_mode import convert_to_list_mode, get_fwd_bwd
dev = "cpu"
image_shape = (128, 128)
voxel_size = (2.0, 2.0)
image_origin = (-127.0, -127.0)
radial_positions = to_device(xp.linspace(-128, 128, 200), dev)
view_angles = to_device(xp.linspace(0, xp.pi, 180, endpoint=False), dev)
radius = 200.0
proj2d = parallelproj.ParallelViewProjector2D(
    image_shape, radial_positions, view_angles, radius, image_origin, voxel_size
)
img = to_device(xp.zeros(proj2d.in_shape, dtype=xp.float32), dev)
img[32:96, 32:64] = 1.0
img_gt = img.copy()
img_fwd = proj2d(img)
img_fwd_back = proj2d.adjoint(img_fwd)
x_start = proj2d._xstart.reshape(-1, 3)
x_end = proj2d._xend.reshape(-1, 3)
mean_data = img_fwd.ravel()
lm, idx_coord_lookup = convert_to_list_mode(x_start, x_end, mean_data, scale_factor=1000.0)

image_shape = (1, 128, 128)
voxel_size = (1.0, 2.0, 2.0)
image_origin = (0.0, -127.0, -127.0)
fwd, bwd = get_fwd_bwd(image_origin, voxel_size, image_shape)

start_detector = idx_coord_lookup[lm[0]]
end_detector = idx_coord_lookup[lm[1]]
img = parallelproj.joseph3d_back(
            start_detector,
            end_detector,
            image_shape,
            image_origin,
            voxel_size,
            xp.ones(start_detector.shape[0], dtype=xp.float32),
        )
plt.imshow(img[0])


In [ ]:
import prd
detectors = []
for i in range(idx_coord_lookup.shape[0]):
    detectors.append(
        prd.Detector(
            id = i,
            x = idx_coord_lookup[i, 0],
            y = idx_coord_lookup[i, 1],
            z = idx_coord_lookup[i, 2]
        )
    )
scanner_information = prd.ScannerInformation(detectors=detectors)
events = []
for i in range(lm.shape[1]):
    events.append(prd.CoincidenceEvent(detector_1_id=int(lm[0,i]), detector_2_id=int(lm[1,i])))
time_block = prd.TimeBlock(id=0, prompt_events=events)

with prd.BinaryPrdExperimentWriter('data.bin') as writer:
    writer.write_header(prd.Header(scanner=scanner_information))
    writer.write_time_blocks(
        (time_block,)
    )


In [ ]:
from io import BytesIO
with open('data.bin', 'rb') as f:
    data = f.read()
    print(len(data))
    with prd.BinaryPrdExperimentReader(BytesIO(data)) as reader:
        # Read header, get number of detectors
        header = reader.read_header()
        N_detectors = len(header.scanner.detectors)
        idx_coord_lookup_2_electric_boogaloo = xp.zeros((N_detectors, 3))
        for i in range(N_detectors):
            idx_coord_lookup_2_electric_boogaloo[i, 0] = header.scanner.detectors[i].x
            idx_coord_lookup_2_electric_boogaloo[i, 1] = header.scanner.detectors[i].y
            idx_coord_lookup_2_electric_boogaloo[i, 2] = header.scanner.detectors[i].z
        # Read time blocks
        lm_2_electric_boogaloo = []
        for time_blocks in reader.read_time_blocks():
            for event in time_blocks.prompt_events:
                prompt = [event.detector_1_id, event.detector_2_id]
                lm_2_electric_boogaloo.append(prompt)

        lm_2_electric_boogaloo = xp.array(lm_2_electric_boogaloo).T


In [ ]:

start_detector = idx_coord_lookup_2_electric_boogaloo[lm_2_electric_boogaloo[0]]
end_detector = idx_coord_lookup_2_electric_boogaloo[lm_2_electric_boogaloo[1]]
img_2_electric_boogaloo = parallelproj.joseph3d_back(
            start_detector,
            end_detector,
            image_shape,
            image_origin,
            voxel_size,
            xp.ones(start_detector.shape[0], dtype=xp.float32),
        )
plt.imshow(img_2_electric_boogaloo[0])


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
fig.colorbar(axs[0].imshow(img[0]), ax=axs[0])
fig.colorbar(axs[1].imshow(img_2_electric_boogaloo[0]), ax=axs[1])
# difference
fig.colorbar(axs[2].imshow(img[0] - img_2_electric_boogaloo[0]), ax=axs[2])
plt.show()
